In [1]:
import numpy as np
import pandas as pd
import gc
IDIR = '../input/'
FEATURES_PATH = './features3/'

In [ ]:
gc.collect()

In [ ]:
print('loading prior')
#priors = pd.read_csv(IDIR + 'order_products__prior.csv', dtype={
#            'order_id': np.int32,
#            'product_id': np.uint16,
#            'add_to_cart_order': np.int16,
#            'reordered': np.int8})


priors = pd.read_hdf(IDIR+"input.h5","priors")

print('loading train')
trains = pd.read_hdf(IDIR+"input.h5","trains")

#trains = pd.read_csv(IDIR + 'order_products__train.csv', dtype={
#            'order_id': np.int32,
#            'product_id': np.uint16,
#            'add_to_cart_order': np.int16,
#            'reordered': np.int8})

print('loading orders')
orders = pd.read_csv(IDIR + 'orders.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32})

print('loading products')
products = pd.read_csv(IDIR + 'products.csv', dtype={
        'product_id': np.uint16,
        'order_id': np.int32,
        'aisle_id': np.uint8,
        'department_id': np.uint8},
        usecols=['product_id', 'aisle_id', 'department_id'])

print('loading aisle')
aisles = pd.read_csv(IDIR + 'aisles.csv', dtype={
        'aisle_id': np.int16 })

print('loading department')
departments = pd.read_csv(IDIR + 'departments.csv', dtype={
        'department_id': np.int16 })
        

print('priors {}: {}'.format(priors.shape, ', '.join(priors.columns)))
print('orders {}: {}'.format(orders.shape, ', '.join(orders.columns)))
print('trains {}: {}'.format(trains.shape, ', '.join(trains.columns)))

In [ ]:
priors.to_hdf(IDIR+"input.h5","priors", mode="w")
trains.to_hdf(IDIR+"input.h5","trains", mode="a")

# USER PRODUCT

## Product number of days since last order

In [ ]:
orders["days_since_first_order"] = orders.groupby('user_id').days_since_prior_order.cumsum()
priors = pd.merge(priors, orders, on=['order_id'], how='left')

In [ ]:
priors.head()

In [ ]:
up_info = priors.groupby(['user_id', 'product_id'])[['days_since_first_order']].max().reset_index()
user_info = orders[orders.eval_set != "prior"][['user_id', 'days_since_first_order']]
user_info.rename(columns={"days_since_first_order": "last_order_days_since_first_order"}, inplace = True)
up_info = pd.merge(up_info, user_info, on="user_id", how="left")

up_info.days_since_first_order = up_info.days_since_first_order.fillna(0)
up_info['up_days_since_last_order'] = (up_info.last_order_days_since_first_order - up_info.days_since_first_order).astype(np.int16)
up_info.drop(['days_since_first_order', 'last_order_days_since_first_order'], axis = 1, inplace=True)


In [ ]:
#up_info.set_index(['user_id', 'product_id']).to_csv(FEATURES_PATH +  "up_days_since_last_order.csv")

In [ ]:
up_info.set_index(['user_id', 'product_id']).to_hdf(FEATURES_PATH+"features.h5", "up_days_since_last_order", mode='a')

In [ ]:
up_info.head()

## Product order rate and number of orders from the last order

In [ ]:
#priors = pd.merge(priors, orders, on="order_id", how="inner")

In [ ]:
user_info = orders[orders.eval_set == "prior"].groupby('user_id') \
        .agg({'order_number': np.max}) \
        .rename(columns={'order_number': 'user_orders'})
        
user_info.reset_index(inplace = True)

In [ ]:
user_info.head()

In [ ]:
up_info = pd.DataFrame(priors.groupby(['user_id', 'product_id']).size().astype(np.int16), columns=['up_orders'])
up_info[['up_first_order', 'up_last_order']] = priors.groupby(['user_id', 'product_id']).order_number.agg([np.min, np.max])
up_info.reset_index(inplace = True)
up_info.user_id = up_info.user_id.astype(np.int32)
up_info.product_id  = up_info.product_id.astype(np.int32)

In [ ]:
up_info = pd.merge(up_info, user_info, on='user_id', how='left')
up_info['up_order_rate'] = (up_info.up_orders/up_info.user_orders).astype(np.float32)
up_info['up_orders_since_last_order'] = (up_info.user_orders - up_info.up_last_order).astype(np.float32)
up_info['up_order_rate_since_first_order'] = (up_info.up_orders / (up_info.user_orders - up_info.up_first_order + 1)).astype(np.float32)

#finish, remove temporal user feature
up_info.drop(['user_orders'], axis=1, inplace=True)

In [ ]:
up_info.head()

In [ ]:
up_info.set_index(['user_id', 'product_id'], inplace = True)
up_info.to_hdf(FEATURES_PATH+"features.h5", "up_order_rates", mode='a')
#up_info.to_csv(FEATURES_PATH +  "up_order_rates.csv")

## Average add to cart order

In [ ]:
up_info = priors.groupby(['user_id', 'product_id'])[['add_to_cart_order']].mean().astype(np.float32)
up_info.rename(columns={"add_to_cart_order":"up_add_to_cart_order_mean"}, inplace=True)

In [ ]:
up_info.head()

In [ ]:
up_info.to_hdf(FEATURES_PATH+"features.h5", "up_add_to_cart_order_mean", mode='a')
#up_info.to_csv(FEATURES_PATH +  "up_add_to_cart_order_mean.csv")

## Average days since prior order, order hour, hour day of week

In [ ]:
priors = pd.merge(priors, orders, on='order_id', how='left')

In [ ]:
order_stat = priors.groupby('order_id').agg({'order_id': 'size'}).rename(columns={'order_id': 'order_size'}).reset_index()

In [ ]:
priors= pd.merge(priors, order_stat, on='order_id')
priors['add_to_cart_order_relative'] = (priors.add_to_cart_order / priors.order_size).astype(np.float32)

In [ ]:
priors.head()

In [ ]:
up_info = priors.groupby(['user_id', 'product_id']).agg({'days_since_prior_order': np.mean,
                                                     'order_dow': np.mean,
                                                     'order_hour_of_day': np.mean,
                                                     'add_to_cart_order_relative': np.mean}).astype(np.float32)

In [ ]:
up_info.rename(columns = {'days_since_prior_order': 'up_days_since_prior_order_mean',
                         'order_dow': 'up_order_dow_mean',
                         'order_hour_of_day': 'up_order_hour_of_day_mean',
                         'add_to_cart_order_relative': 'up_add_to_cart_order_relative_mean'}, inplace = True)

In [ ]:
up_info.head()

In [ ]:
up_info.up_days_since_prior_order_mean.fillna(0, inplace = True)

In [ ]:
up_info.dtypes

In [ ]:
up_info.to_hdf(FEATURES_PATH+"features.h5", "up_info", mode='a')

## Reordered in the train set

In [ ]:
#must reset

In [ ]:
trains = pd.merge(trains, orders[['order_id', 'user_id']], on = 'order_id', how='left')
priors = pd.merge(priors, orders[['order_id', 'user_id']], on="order_id", how="inner")
up_info = pd.DataFrame(priors.groupby(['user_id', 'product_id']).size()).reset_index()
up_info.drop([0], axis = 1, inplace = True)
up_info = pd.merge(up_info, trains[['user_id', 'product_id', 'reordered']], on=['user_id', 'product_id'], how='left')
up_info.reordered.fillna(0, inplace = True)
up_info.rename(columns={"reordered": "up_reordered"}, inplace=True)

In [ ]:
up_info.head()

In [ ]:
up_info.set_index(['user_id','product_id'], inplace=True)
up_info.to_hdf(FEATURES_PATH+"features.h5", "up_reordered", mode='a')
#up_info.to_csv(FEATURES_PATH + "up_reordered.csv")

# USER

In [ ]:
#priors = pd.merge(priors, orders[['order_id', 'user_id']], on="order_id", how="inner")

In [ ]:
user_info = orders[orders.eval_set == "prior"].groupby('user_id') \
        .agg({'order_number': np.max}) \
        .rename(columns={'order_number': 'user_total_order'})        
user_info.reset_index(inplace = True)

In [ ]:
user_reorder_rate = priors.groupby(['user_id']).reordered.agg([np.sum, np.size])
user_reorder_rate['user_reorder_rate'] = (user_reorder_rate['sum']/user_reorder_rate['size']).astype(np.float32)
user_reorder_rate.reset_index(inplace=True)
user_info = pd.merge(user_info, user_reorder_rate[['user_id', 'user_reorder_rate']], on="user_id", how='left')
del user_reorder_rate

In [ ]:
orders_count = priors.groupby('order_id')['product_id'].count()
orders['item_count'] = orders['order_id'].map(orders_count)
user_basket_avg = orders[orders.eval_set=="prior"].groupby('user_id')[['item_count']].mean()
user_basket_avg.rename(columns={"item_count": "user_order_size_mean"}, inplace=True)
user_basket_avg.reset_index(inplace=True)
orders.drop(['item_count'], axis = 1, inplace = True)

user_info = pd.merge(user_info, user_basket_avg, on="user_id", how='left')
del user_basket_avg

In [ ]:
user_info.set_index('user_id').to_hdf(FEATURES_PATH+"features.h5", "user_info", mode='a')

## last order info

In [ ]:
user_info = pd.read_hdf(FEATURES_PATH+"features.h5", "user_info")

In [ ]:
user_info.reset_index(inplace = True)

In [ ]:
last_orders_info = orders[orders.eval_set != 'prior'][['user_id',  'order_dow', 'order_hour_of_day', 'eval_set', 'days_since_prior_order']]
user_info = user_info.merge(last_orders_info, on='user_id', how='left')
user_info.rename(columns = {
    'order_dow': 'user_order_dow',
    'order_hour_of_day': 'user_order_hour_of_day',
    'eval_set':'user_eval_set', 
    'days_since_prior_order': 'user_days_since_prior_order'
}, inplace = True)

In [ ]:
user_info.head()

## User mean days since prior

In [ ]:
user_orders = orders[orders['eval_set'] == "prior"].groupby(['user_id'])['order_number'].max().to_frame('user_orders')
user_orders['user_period'] = orders[orders['eval_set'] == "prior"].groupby(['user_id'])['days_since_prior_order'].sum()
user_orders['user_days_since_prior_mean'] = orders[orders['eval_set'] == "prior"].groupby(['user_id'])['days_since_prior_order'].mean()

In [ ]:
#concatenate
user_info = pd.read_hdf(FEATURES_PATH+"features.h5", "user_info")
user_info['user_days_since_prior_mean']=  user_orders['user_days_since_prior_mean'].astype(np.float32)
user_info.to_hdf(FEATURES_PATH+"features.h5", "user_info", mode='a')

### Average dow, hod

In [ ]:
user_orders = orders[orders['eval_set'] == "prior"].groupby(['user_id']).agg({
    "order_dow": np.mean,
    "order_hour_of_day": np.mean}).astype(np.float32).reset_index()

In [ ]:
user_orders.rename(columns = 
                   {"order_dow":"user_order_dow_mean", 
                    "order_hour_of_day":"user_order_hour_of_day_mean"}, inplace = True)

In [ ]:
user_info = pd.read_hdf(FEATURES_PATH+"features.h5", "user_info").reset_index()
user_info = user_info.merge(user_orders, on ="user_id", how="left")
user_info.set_index('user_id').to_hdf(FEATURES_PATH+"features.h5", "user_info", mode='a')

In [ ]:
user_info.isnull().sum()

# PRODUCT

In [ ]:
product_reorder_sum = priors.groupby('product_id').reordered.sum()
product_reorder_size = priors.groupby('product_id').reordered.size()
product_reorder_ratio = ((product_reorder_sum+1)/(product_reorder_size + 1)).astype(np.float32)
product_info = pd.DataFrame(product_reorder_ratio)
product_info.rename(columns={"reordered": "product_reorder_ratio"}, inplace=True)
product_info.reset_index(inplace = True)
product_info = pd.merge(product_info, products, on="product_id", how="inner")

In [ ]:
product_info.head()

In [ ]:
product_info.sort_values(by="product_reorder_ratio")

In [ ]:
product_info.set_index('product_id').to_hdf(FEATURES_PATH+"features.h5", "product_info", mode='a')
#product_info.set_index('product_id').to_csv(FEATURES_PATH +  "product_info.csv")

## Product reorder probability

In [ ]:
priors = priors.merge(orders, on="order_id", how="left")
priors.sort_values(['user_id', 'order_number', 'product_id'], ascending=True, inplace=True)
priors["product_time"] = priors.groupby(['user_id', 'product_id']).cumcount()+1


In [ ]:
sub1 = priors[priors['product_time'] == 1].groupby('product_id').size().to_frame('prod_first_orders')
sub1['prod_second_orders'] = priors[priors['product_time'] == 2].groupby('product_id').size().to_frame('prod_second_orders')
sub1.prod_second_orders.fillna(0, inplace = True)
sub1['prod_reorder_probability'] = sub1['prod_second_orders']/sub1['prod_first_orders']

In [ ]:
product_info = pd.read_hdf(FEATURES_PATH+"features.h5", "product_info")
product_info['product_reorder_probability']  = sub1['prod_reorder_probability'].astype(np.float32)
product_info.to_hdf(FEATURES_PATH+"features.h5", "product_info", mode= "a")

In [ ]:
product_info = pd.read_hdf(FEATURES_PATH+"features.h5", "product_info")

In [ ]:
product_info.dtypes

### Is organic

In [3]:
products = pd.read_csv(IDIR + 'products.csv', dtype={
        'product_id': np.uint16,
        'order_id': np.int32,
        'aisle_id': np.uint8,
        'department_id': np.uint8})

In [4]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [10]:
products['is_organic'] = products.product_name.str.contains('organic', case=False, na = False)

In [11]:
products['is_organic'].value_counts()

False    44652
True      5036
Name: is_organic, dtype: int64

In [13]:
products.set_index('product_id', inplace=True)

In [14]:
product_info = pd.read_hdf(FEATURES_PATH+"features.h5", "product_info")
product_info['is_organic']  = products['is_organic'].astype(np.uint8)
product_info.to_hdf(FEATURES_PATH+"features.h5", "product_info", mode= "a")

In [15]:
product_info.head()

,product_reorder_ratio,aisle_id,department_id,product_reorder_probability,is_organic
product_id,,,,,
1,0.613600,61,19,0.385475,0
2,0.142857,104,13,0.102564,0
3,0.733813,94,7,0.486486,0
4,0.448485,38,1,0.351648,0
5,0.625000,5,13,0.666667,0


# AISLE DEPARTMENT

In [ ]:
#need reset

In [ ]:
priors = pd.merge(priors, orders[['user_id', 'order_id','order_number']], on='order_id', how='left')
priors = pd.merge(priors, products, on='product_id', how='left')

In [ ]:
aisle_reordered_sum = priors.groupby('aisle_id').reordered.sum()
aisle_reordered_size = priors.groupby('aisle_id').reordered.size()
aisle_reordered_ratio = ((aisle_reordered_sum + 1)/(aisle_reordered_size + 1)).astype(np.float32)
aisle_info = pd.DataFrame(aisle_reordered_ratio)
aisle_info.rename(columns={"reordered": "aisle_reorder_ratio"}, inplace=True)

In [ ]:
aisle_info.head()

In [ ]:
aisle_info.sort_values(by="aisle_reorder_ratio")

In [ ]:
aisle_info.to_hdf(FEATURES_PATH+"features.h5", "aisle_info", mode='a')
#aisle_info.to_csv(FEATURES_PATH + "aisle_info.csv")

In [ ]:
department_reordered_sum = priors.groupby('department_id').reordered.sum()
department_reordered_size = priors.groupby('department_id').reordered.size()
department_reordered_ratio = ((department_reordered_sum + 1)/(department_reordered_size + 1)).astype(np.float32)
department_info = pd.DataFrame(department_reordered_ratio)
department_info.rename(columns={"reordered": "dep_reorder_ratio"}, inplace=True)

In [ ]:
department_info.head()

In [ ]:
department_info.to_hdf(FEATURES_PATH+"features.h5", "department_info", mode='a')
#department_info.to_csv(FEATURES_PATH + "department_info.csv")

## User Aisle

In [ ]:
user_aisle_reordered_sum = priors.groupby(['user_id', 'aisle_id']).reordered.sum()
user_aisle_reordered_size = priors.groupby(['user_id', 'aisle_id']).reordered.size()
user_aisle_reordered_ratio = ((user_aisle_reordered_sum + 1)/(user_aisle_reordered_size + 1)).astype(np.float32)
user_aisle_info = pd.DataFrame(user_aisle_reordered_ratio)
user_aisle_info.rename(columns={"reordered": "user_aisle_reordered_ratio"}, inplace=True)

In [ ]:
user_aisle_info.head()

In [ ]:
user_aisle_info.sort_values(by="user_aisle_reordered_ratio").head()

In [ ]:
print(user_aisle_info.isnull().sum())

In [ ]:
user_aisle_info.to_hdf(FEATURES_PATH+"features.h5", "user_aisle_info", mode='a')
#user_aisle_info.to_csv(FEATURES_PATH + "user_aisle_info.csv")

### User aisle ratio (proportion)

In [ ]:
#reset
priors = pd.merge(priors, orders[['user_id', 'order_id']], on='order_id', how='left')
priors = pd.merge(priors, products, on='product_id', how='left')

In [ ]:
user_aisle = priors.groupby(['user_id', 'aisle_id']).size().to_frame("user_aisle_count").reset_index()
user_prod_count = priors.groupby(['user_id']).size()
user_aisle['user_prod_count'] = user_aisle['user_id'].map(user_prod_count)
user_aisle['user_aisle_ratio'] = (user_aisle['user_aisle_count']/ user_aisle['user_prod_count']).astype(np.float32)
user_aisle.set_index(['user_id', 'aisle_id'], inplace = True)

In [ ]:
#merge
user_aisle_info = pd.read_hdf(FEATURES_PATH+"features.h5", "user_aisle_info")
user_aisle_info['user_aisle_ratio'] = user_aisle['user_aisle_ratio']
user_aisle_info.to_hdf(FEATURES_PATH+"features.h5", "user_aisle_info", mode ="a")

In [ ]:
user_aisle_info.head()

## User Department

In [ ]:
user_dep_reordered_sum = priors.groupby(['user_id', 'department_id']).reordered.sum()
user_dep_reordered_size = priors.groupby(['user_id', 'department_id']).reordered.size()
user_dep_reordered_ratio = ((user_dep_reordered_sum + 1)/(user_dep_reordered_size + 1)).astype(np.float32)
user_dep_info = pd.DataFrame(user_dep_reordered_ratio)
user_dep_info.rename(columns={"reordered": "user_dep_reordered_ratio"}, inplace=True)

In [ ]:
user_dep_info.head()

In [ ]:
user_dep_info.sort_values(by="user_dep_reordered_ratio").head()

In [ ]:
user_dep_info.isnull().sum()

In [ ]:
user_dep_info.to_hdf(FEATURES_PATH+"features.h5", "user_dep_info", mode='a')
#user_dep_info.to_csv(FEATURES_PATH + "user_dep_info.csv")

### User department ratio (proportion)

In [ ]:
user_dep = priors.groupby(['user_id', 'department_id']).size().to_frame("user_dep_count").reset_index()
user_prod_count = priors.groupby(['user_id']).size()
user_dep['user_prod_count'] = user_dep['user_id'].map(user_prod_count)
user_dep['user_dep_ratio'] = (user_dep['user_dep_count']/ user_dep['user_prod_count']).astype(np.float32)
user_dep.set_index(['user_id', 'department_id'], inplace = True)

In [ ]:
user_dep.head()

In [ ]:
#merge
user_dep_info = pd.read_hdf(FEATURES_PATH+"features.h5", "user_dep_info")
user_dep_info['user_dep_ratio'] = user_dep['user_dep_ratio']

In [ ]:
user_dep_info

# Test